In [10]:
from dotenv import dotenv_values
import praw
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [11]:
# Download the VADER lexicon
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\seb\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [12]:
# Load environment variables
env = dotenv_values('.env')

In [13]:
# Set up Reddit API credentials
reddit = praw.Reddit(
    client_id=env['R_CLIENT_ID'],
    client_secret=env['R_CLIENT_SECRET'],
    user_agent=env['R_USER_AGENT']
)

In [14]:
subreddit_name = 'Economics+economy+stocks'
subreddit = reddit.subreddit(subreddit_name)

In [15]:
# Initialize the sentiment analyzer
sia = SentimentIntensityAnalyzer()

praw.models.Submission: 
https://praw.readthedocs.io/en/v7.7.1/code_overview/models/submission.html

In [ ]:
import database

In [16]:
import time

# Fetch submissions with pagination
submissions = []
last_submission = None

while True:
    new_submissions = list(subreddit.new(limit=1000, params={'after': last_submission}))
    if not new_submissions:
        break
    submissions.extend(new_submissions)
    last_submission = new_submissions[-1].fullname
    time.sleep(2)  # Sleep to avoid hitting rate limits

# Process submissions
for submission in submissions:
    print(f"Title: {submission.title}, Created: {submission.created_utc}")

Title: A ‘hidden liability’: Colleges face up to $950B in capital needs, Moody’s says, Created: 1724818174.0
Title: African migrant delivery workers wait for orders in Hoboken, New Jersey , Created: 1724816405.0
Title: Cramer ruins everything (SBUX), Created: 1724815394.0
Title: how much does your brokerage choice really matter?, Created: 1724814908.0
Title: You Can Just Peel Open A Cybertruck Like An Orange, Created: 1724807216.0
Title: US consumer confidence rises in August as Americans’ optimism about future improves, Created: 1724806634.0
Title: The Cheapest New Cars of 2024, Created: 1724805914.0
Title: Paying money to get your own cash: Some big retailers charge for cash-back requests, federal agency finds, Created: 1724801627.0
Title: Xpeng Releases Mass-Market EV with 300-Mile Range Starting at $16,800, Created: 1724801433.0
Title: Trump budget would spike deficits by nearly 5 times more than Harris proposal, says Penn Wharton, Created: 1724798486.0
Title: Trump budget would sp

In [ ]:
submissions_data = []  # submission data store

for submission in subreddit.stream.submissions():
    if submission.stickied:  # if post is pinned, skip it
        continue

    sentiment_title = sia.polarity_scores(submission.title)
    sentiment_title_vader = sentiment_title['compound']
    sentiment_selftext = sia.polarity_scores(submission.selftext)
    sentiment_selftext_vader = sentiment_selftext['compound'] if submission.selftext else None

    submissions_data.append({
        'id': submission.id,
        'created_utc': int(submission.created_utc),
        'subreddit': submission.subreddit.display_name,
        'author_name': submission.author.name,
        'title': submission.title,
        'selftext': submission.selftext,
        'url': submission.url,
        'num_comments': submission.num_comments,
        'score': submission.score,
        'upvote_ratio': submission.upvote_ratio,
        'sentiment_title_vader': sentiment_title_vader,
        'sentiment_selftext_vader': sentiment_selftext_vader
    })

    #database.insert_row('sentiment', submission_data)

    print(submissions_data[-1])
    print(len(submissions_data))
    for comment in submission.comments:
        pass
        #print(comment.selftext)